## Load and initialize


settings


In [ ]:
# sample number

# j = '_arcsinh'+'18.2'+'a'# '18.2'+'a'#
# j = '18'+'a'#  '_arcsinh'+'18.2'+'a'#'14.2'+'a'#'_raw'+'14.2'+'a'#
            
# '7.1a','13a','14a','15a','17a','18a','19a','20a','8.1a'

# j = '13'+'a'
# j = 'b2345'+'a'
# j = '5a'
# j = '18.2'+'a'
# j = 'b2345'+'s'#stroma - exists only in batch

# j = 'b2345'+'a'
# j = 'b2345'+'s'+'_prescaled'
args = {
        'j' : j,
        'feautures_ind':2,
        'data_folder' : 'Data_',
        # 'data_folder' : 'age_prescaled',
        # create visualization (umap, dbscan are always shown)
        # 'visualize':True,'plotUMAP' : True,
        
        # show figures in notebook
        'show' : True, 'saveSVG' :False,
        'recalculate_umap' : False,# calculate umap
        'recalculate_db' : False,# calculate dbscan
        'create_adjusted' : False,
        'compare' : False,# compare 11,stroma and rest of the cells
             
        }
# create_adjusted = False
# compare = False# compare 11,stroma and rest of the cells
# print(j, 'ind:',config['feautures_ind'], dir_data1, config, create_adjusted)

# parent_dir = '/home/yishai/breast_cancer_PHD_research'



use_MASTER = True





%run imports.ipynb # import all imported packages from imports.ipynb

samples = Series(df['samp'])
uniq_samples = samples.unique()# don't drop anchor samples
config['dir_data']

log into file

In [ ]:
import  print_n_log  
print = print_n_log.run('log', f'{config["dir_plots"]}/logs.log', 'DEBUG')
# try:
#     print = original_print#if print function was saved in original_print  - return it to print - happens in the second run
# except:
#     original_print = print#if print function was not saved in original_print  - save it in original_print (for future use) - happens in the first run
#     print = print_n_log.run('log', f'{config["dir_plots"]}/logs.log', 'DEBUG')


##  clustering

UMAP

plot for each feature according to its intensity


In [ ]:
# %run umap.ipynb   
from clustering import *
clustering = Clustering(**config)

from plots import *
plot = Plots(**config)#build Umap_dbscan class (either from MASTER or from here) using parent class containg the config data


In [ ]:

umapData = clustering.umap(df)
plot.umap(umapData,df['H4'].copy(),
            title = f'{config["title"]} UMAP ',
            figname = '1_'+config["figname"]+'UMAP_(core)')



In [ ]:

# samples = samples[(samples==19)].copy()
# umapData = umapData.loc[samples.index].copy()
# df = df.loc[samples.index].copy()
# uniq_samples = samples.unique()# don't drop anchor samples

# print(samples.unique(), len(umapData),)

# config['j'] = '19'
# config["title"] = 'samp19.1a CellIden+EpiCols Based '
# config['figname'] = 'samp19.1a_CellIden+EpiCols_Based_'

# if len(uniq_samples)==1:
#     config['isSingleSample'] = True


# # %run umap.ipynb   
# from clustering import *
# clustering = Clustering(**config)
# clustering.umapData = umapData
# from plots import *
# plot = Plots(**config)#build Umap_dbscan class (either from MASTER or from here) using parent class containg the config data

# plot.umap(umapData,df['H4'].copy(),
#             title = f'{config["title"]} UMAP ',
#             figname = '1_'+config["figname"]+'UMAP_(core)')



Dbscan


In [ ]:
labels = clustering.DBscan(cluster_colors())#,params = [0.15,50]
# from plot_functions import drawDbscan
plot.DBscan(labels,clustering.dbData,clustering.colors,
            title=f'{config["title"]} DBscan ',
            figname='1_'+config["figname"]+'DBscan')


Dbscan without noise

In [ ]:
plot.DBscan(labels[labels!=-1],clustering.dbData,clustering.colors,
            title=f'{config["title"]} DBscan ',
            figname='1_'+config["figname"]+'DBscan_without_noise')

In [ ]:
labels.index = df.index

todo: plot Clusters

In [ ]:
# from plot_functions import *
# # if config['visualize'] and config['plotUMAP']:
# plotClusters(df, umapData, labels, clustering.colors, config['groups']['NamesAll'], config,
#             title='Clusters ' + config['title'],
#             figname='1_'+config['figname']+'Clusters')

load single samples labels from file

## classification

In [ ]:
# %run classes.ipynb
from classes import *
classification = Classification(**config)#build class (either from MASTER or from here) using parent class containg the config data
labels_to_classes_dict = classification.classes_dict(f"{classification.dir_data}_clusters.csv")



assign classes

load csv of class for every cluster in every sample - and assign to every data point in every sample

In [ ]:
# if not config['isSingleSample']:
# sample_labels = Series(index = df.index)#initialize as none
classes = Series(index = df.index)#initialize as none
for sampNum in uniq_samples:
    print(sampNum)
    try:#load classes from file
        classes.loc[df.index[df['samp'] == sampNum]] = clustering.pickle_load(f"classes_samp{f'{sampNum}a'}_{config['name']}_Based_", config['dir_data'])
    except:#calculate classes
        print(f'{sampNum} class calculated from dbLabels_sample')
        sample_labels = Series(clustering.pickle_load(f"dbLabels_samp{f'{sampNum}a'}_{config['name']}_Based_", config['dir_data']),     index = df.index[df['samp'] == sampNum])
        classes.loc[df.index[df['samp'] == sampNum]] = classification.get_classes(samples[samples==sampNum],sample_labels,labels_to_classes_dict)
    
del sampNum
# else:
#     sample_labels = labels

print (f'unique classes: {sorted(classes.unique())}')
print ('classification - done')

In [ ]:
# classes

create Basal like 2 class

In [ ]:
if not config['isSingleSample'] and 'Basal-like2' not in classes.unique():
    classes1 = classes.copy()
    classes1.loc[umapData[umapData['umap2']>4.12].index] = 'Basal-like2'
    classes1[(classes=='Cycling') & (classes1=='Basal-like2')] = 'Cycling-Basal-like2'
    # classes1 = classes1[classes1!='Noise']
    classes1[(classes1!='Cycling') & (classes1!='Basal-like2')& (classes1!='Cycling-Basal-like2')&(classes1!='Noise')] = 'Rest'
    classes = classes1
    # -----------
    # df = df.loc[classes.index].copy()
    # samples = samples.loc[classes.index].copy()
    # labels = labels.loc[classes.index].copy()
    # umapData = umapData.loc[samples.index].copy()
    uniq_samples = samples.unique()# don't drop anchor samples
    
    
    # # (classes1.index == sample_labels.index).all()
    # for sampNum in uniq_samples:
    #     print(sampNum)
    #     print(f"classes_samp{f'{sampNum}a'}_{config['name']}_Based_")
    #     clustering.pickle_dump(f"classes_samp{f'{sampNum}a'}_{config['name']}_Based_",classes.loc[samples.index[samples == sampNum]].to_numpy(),  config['dir_data'])
    #     # = clustering.pickle_load(f"dbLabels_samp{f'{sampNum}a'}_{config['name']}_Based_", config['dir_data'])


plot by class

In [ ]:
plot.labeled_umap(umapData,classes.copy(),colors = class_colors(),
                    title=config['title']+f' UMAP - classes',
                    figname='4_'+config['figname']+f'UMAP_by_classes')

bimodal gassian fit

In [ ]:

# if '.2a' in config['j'] and config['feautures_ind'] == 3:
    
#     labels = None# verify that the labels from previous are not used
#     if '_arcsinh' in j:
#         from gaussian_fit import *


#         x, y = sns.kdeplot(k['pRB'].copy(),label='label').lines[0].get_data()
#         y =  np.asarray(y)/.2 if '18.2' in j else  np.asarray(y)**.5 #highlight the gaussians diffrence
#         # plt.plot(x,y)
#         dir,show,_ = settings;saveSVG = True
#         cutoff = bimodal_minima(x,y, maxfev = 5000,settings =(dir,show,saveSVG)  ,manual_minima =None)#manual_minima =3.70


#         labels = np.zeros(len(k)).astype(int)
#         labels[k['pRB']>cutoff] = 1

#         pickle_dump(f'Labels_{figname}', labels, dir_data)
#     names = ['pRB_low','pRB_high']    
#     try:
#         labels = pickle_load(f'Labels_samp_arcsinh{j}_CellCycle_Based_', dir_data) 
        
#         colors = draw_clusters(data = umapData.copy(),labels = labels,title='cutoff clusters',figname='cutoff_clusters',settings = settings,names =names.copy()) 
#     except:print('for clusters plot: change from -arcsin to processed data')
      


#     # k['Clust'] = [names[l] for l in labels]#labels
#     # labels = None

kmeans

In [ ]:

if config['isCellCycle'] and config['feautures_ind'] == 3:
    from clustering_functions import *

# if '.2a' in j and config['feautures_ind'] == 3:
    if '_arcsinh' in config['j']:
        print('arcsinh')
        # high prb
        labels = kmeans_fit(np.hstack ((umapData.copy(),df[[ 'pRB']].copy()))
                            ,n_clusters=2,n_init=10,max_iter=300)
        colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters',settings = settings,
                names =['pRB- ','pRB+']  )
        ind,_ = highClust_ind(k['pRB'],labels)


        # high CyclinB1
        labels1 = kmeans_fit(df[['CyclinB1']].loc[ind].copy(),n_clusters=2,n_init=10,max_iter=300)
        ind11,ind2 = highClust_ind(df.loc[ind]['CyclinB1'],labels1)
        labels[ind11] = 2#labels1[ind2]+2

        # high H3S28p
        labels2 = kmeans_fit(df[['H3S28p']].loc[ind].copy(),n_clusters=2,n_init=10,max_iter=300)
        ind12,ind2 = highClust_ind(df.loc[ind]['H3S28p'],labels2)
        labels[ind12] = 3#labels2[ind2]+4

        ind13= [i for i in ind12 if i not in ind11]
        labels[ind13] = 4
        
    # load arcsin data
      
        colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters',settings = settings,
                names =['pRB+ not M&G2','pRB- (GO)','pRB+ G2','pRB+ M','pRB+ G2&M']  )
    
    
        labels[ind12] = 2#labels2[ind2]+2
        pickle_dump(f'kmeanLabels_{figname}', labels, dir_data)
        # pickle_dump(f'kmeanColors_{figname}', colors, dir_data)
        labels,labels2 = None,None
    names =['not M&G2','pRB- (GO)','M&G2']
    labels = pickle_load(f'kmeanLabels_samp_arcsinh14.2a_CellCycle_', dir_data) 
    colors = pickle_load(f'kmeanColors_samp_arcsinh14.2a_CellCycle_', dir_data)
    colors = draw_clusters(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
                names =names)   
    
    k['Clust'] = [names[l] for l in labels]#labels



In [ ]:
# from plot_functions import *
 
# # colors = draw_kmeans(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
# #             names =names,crossLabels = [labels__,1,None])

# for i in range (3):
#     print(i)
#     draw_kmeans(data = umapData.copy(),labels = labels,title='kmeans all clusters',figname='kmeans_all_clusters1',settings = settings,
#             names =names,crossLabels = [labels__,1,i])



adjusted dataset


In [ ]:

# from plot_functions import *
    

# # create_adjusted = True
# if config['feautures_ind']==2 and config['create_adjusted']: #(  or ( '.2' in j))
#     if ('b' in j and 'a' not in j):
#         # remove -1 cluster
#         indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
#                             clustersToRemove=[],limits = (-1.8,None,None,None),drawFunc = drawDbscan,contained_samples = False)
#         k = k.copy().reset_index(drop = True).loc[idx_adj];dbscanData, labels,core_samples_mask = dbscanData[idx_adj], labels[idx_adj],core_samples_mask[idx_adj]
        
#         indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
#                             clustersToRemove=[],limits = (None,2,None,None),drawFunc = drawDbscan,contained_samples = True)
#     elif ('.2' in j):
#         # remove -1 cluster
#         indArr,idx_adj,idx_stroma = createAdjustedDataset(k,j,dbscanData, labels,core_samples_mask,settings,figname,
#                             clustersToRemove=[],limits = (-1,None,None,None),drawFunc = drawDbscan,contained_samples = False)
        

#     for kInd,ind in indArr.items():
#         print(f'samp:{kInd}, len:{len(ind)}')     
#         pickle_dump(f"{kInd}_indexes", ind,dir_index)
                            
#     for i,idx in zip(['a','s'],[idx_adj,idx_stroma]):
#         kInd = f'{j}{i}'
#         newK = k.copy().reset_index(drop = True).loc[idx]
#         dict = None;dict ={}
#         dict['k'] = newK.copy()
#         pickle_dump(f'k{kInd}_dict', dict,dir_data)
#         pickle_dump(f'k{kInd}_names_dict', names_dict,dir_data)
#     # print(len(k))
#     # print(len(newK))
#     # k = newK.copy()
#     # CAll = pd.concat([k.copy()])


      
#     indArr = None
#     # K = pickle_load(f'k{sampNum_}_dict', dir_data)['k'].reset_index(drop=True)
#     # ind_samp = [i for i, j in zip(k['ind'],k['samp'] == sampNum) if j]
#     # ind_samp = [ K.index[K['ind']==i][0] for i in ind_samp ]
#     # 7933


## HeatMap


In [ ]:
from heatmap import *
heatmaps = Heatmap(**config)#build class (either from MASTER or from here) using parent class containg the config data


HeatMap by clusters


In [ ]:
for features_group in heatmaps.features_groups:# CellCycle features are present only in the '.2' data other wise only epigenetic and cellcycle features
    heatmaps.plot(  df.copy(), labels.copy(),
                    features = config['groups'][features_group].copy(),
                    ind = list(df.index[labels != -1]),#remove unclustered -1
                    title='HeatMap clusters -' + config['title'] + ': '+features_group,
                    figname='3_HeatMap_clusters_'+config['figname']+'using_'+features_group,)
    
print('HeatMap by clusters - done')   


HeatMap by sample

In [ ]:
if not config['isSingleSample']:
    for features_group in heatmaps.features_groups:# CellCycle features are present only in the '.2' data other wise only epigenetic and cellcycle features  
        heatmaps.plot(  df.copy(), samples.copy(),
                        features = config['groups'][features_group].copy(),
                        # ind = list(df.index[labels != -1]),#remove unclustered -1
                        title='HeatMap samples -' + config['title'] + ': '+features_group,
                        figname='3_HeatMap_samples_'+config['figname']+'using_'+features_group,)
        del features_group
        print('HeatMap by sample - done')

Heatmap by class

In [ ]:
if not config['isSingleSample']:
    for features_group in heatmaps.features_groups:# CellCycle features are present only in the '.2' data other wise only epigenetic and cellcycle features  
        heatmaps.plot(  df.copy(), classes.copy(),
                        features = config['groups'][features_group].copy(),
                        # ind = list(df.index[labels != -1]),#remove unclustered -1
                        title='HeatMap classes -' + config['title'] + ': '+features_group,
                        figname='3_HeatMap_classes_'+config['figname']+'using_'+features_group,)
        del features_group
        print('HeatMap by sample - done')

Heatmap by sample and class

In [ ]:
# if not config['isSingleSample']:
ind = classes.copy()[classes!='noise'][classes!='unknown'][classes!='classification_error'].index

samp_class = Series(samples.astype(str)+'_'+classes.astype(str))
heatmaps.pickle_dump(f'samp_class_{config["name"]}', samp_class, config['save_add'])
heatmaps.pickle_dump(f'classes_{config["name"]}', classes, config['save_add'])
heatmaps.pickle_dump(f'df_{config["name"]}', df, config['save_add'])


for features_group in heatmaps.features_groups:# CellCycle features are present only in the '.2' data other wise only epigenetic and cellcycle features
    heatmaps.plot(  df.copy().loc[ind], samp_class.copy().loc[ind],features = config['groups'][features_group].copy(),
                    # ind = list(df.index[labels != -1]),#remove unclustered -1
                    title='HeatMap samples and class - ' + config['title'] + ': '+features_group,
                        figname='3_HeatMap_samples_and_classes'+config['figname']+'using_'+features_group,)
del samp_class,features_group, ind
print('heatmap by sample and and class - done')


todo: heatmap by comparing 11,stroma and rest of the cells (in heatmaps notebook)

MeanDist - in heatmaps notebook (currently not used)


## advance umap plots


all samples by color in mutual umap figure 

(without anchor samples)

In [ ]:
samples_without_repeats = samples.loc[~samples.copy().astype(str).str.contains('.1')]# drop anchor samples
plot.labeled_umap(umapData,samples_without_repeats.copy(),colors = sample_colors(),
                    title = f'{config["title"]} UMAP - by_sample=ALL',
                    figname='4_'+config["figname"]+f'UMAP_sampleALL_by_sample')
del samples_without_repeats
print('UMAP all samples by color in mutual umap figure  - done')

each single sample by color in mutual umap figure 


In [ ]:
for sampNum in sorted(samples.unique()):
    single_sample = samples.loc[samples[samples == sampNum].index].copy()
    plot.labeled_umap(umapData,single_sample.copy(),
                        title = f'{config["title"]} UMAP - by_sample={sampNum}',
                        figname='4_'+config["figname"]+f'UMAP_sample{sampNum}_by_sample')
del single_sample
print('UMAP by single sample in mutual umap figure  - done')

each mutual cluster's samples by color in the mutual umap

how a single cluster is with diffrent clusters

In [ ]:
# for clustNum in sorted(labels.unique()):
#     single_cluster_samples = samples.loc[labels[labels == clustNum].index].copy()
#     plot.labeled_umap(umapData,single_cluster_samples.copy(),
#                         title = f'{config["title"]} UMAP - by_cluster={clustNum}',
#                         figname='4_'+config["figname"]+f'UMAP_cluster{clustNum}_by_sample')
#     print('UMAP by mutual cluster samples  - done')
# del single_cluster_samples
# print('UMAP by mutual cluster samples  - done')

plot each single sample using its single sample clusters in mutual umap

In [ ]:
# if config['visualize'] :#plot on adjusted
if not config['isSingleSample']:

    for sampNum in sorted(samples.loc[~samples.copy().astype(str).str.contains('.1')].unique()):# drop anchor samples:

        single_sample_labels = sample_labels.loc[samples [samples== sampNum].index].copy()#clusters of specific sample
        # plot.labeled_umap_by_sample(umapData,  single_sample_labels.copy(),
        #                         title=config['title']+f' UMAP - by_sample={sampNum} clusters',
        #                         figname='4_'+config['figname']+f'UMAP_sample{sampNum}_by_sample_clusters')
        plot.labeled_umap(umapData,single_sample_labels.copy(),
                                title=config['title']+f' UMAP - by_sample={sampNum} clusters',
                                figname='4_'+config['figname']+f'UMAP_sample{sampNum}_by_sample_clusters')
        del single_sample_labels,sampNum
    print('single sample using its single sample clusters - done')

each single samples classes in mutual umap


In [ ]:

for sampNum in sorted(samples.unique()):
    

    single_sample_classes = classes.loc[samples [samples== sampNum].index].copy()#clusters of specific sample
    plot.labeled_umap(umapData,single_sample_classes.copy(),colors = class_colors(),
                        title=config['title']+f' UMAP - by_sample={sampNum} classes',
                        figname='4_'+config['figname']+f'UMAP_sample{sampNum}_by_sample_classes_with_noise')
    
    # drop noise and unknowns
    single_sample_classes = single_sample_classes[(single_sample_classes!='Noise')&(single_sample_classes!='Unknown')]


    plot.labeled_umap(umapData,single_sample_classes.copy(),

                            title=config['title']+f' UMAP - by_sample={sampNum} classes',colors = class_colors(),
                            figname='4_'+config['figname']+f'UMAP_sample{sampNum}_by_sample_classes')
del single_sample_classes
    

## Percentage

class sample's percentage and sample class's percentage (inverse)

In [ ]:
cluster_samples_probability(samples,classes,config,fname = 'samples_internal_classes_probability',cluster_type_name='sample')
print("UMAP sample class's probability  - done")
if not config['isSingleSample']:# no meaning for single sample as 
    cluster_samples_probability(classes,samples,config,fname = 'classes_samples_probability',cluster_type_name='class')
    print("UMAP class sample's probability  - done")

class cluster's percentage:
    with noise class (noise of the batch clustering) 
    without noise class (noise of the batch clustering) i.e with cluster -1 which comes from each of the sampl's clustering
cluster class's percentage (inverse)

In [ ]:
cluster_samples_probability(labels,classes,config,fname = 'class_prob',cluster_type_name='cluster')

cluster_samples_probability(labels[classes!='noise'],classes[classes!='noise'],config,fname = 'class_prob_without_noise',cluster_type_name='cluster')
print("UMAP cluster classification's probability  - done")
   
   
cluster_samples_probability(classes,labels,config,fname = 'cluster_class_prob',cluster_type_name='class') 

sample cluster's percentage and cluster sample's percentage (inverse)

In [ ]:
cluster_samples_probability(labels,samples,config,fname = 'cluster_samples_probability',cluster_type_name='cluster')
cluster_samples_probability(samples,labels,config,fname = 'sample_cluster_probability',cluster_type_name='sample')

print("UMAP cluster sample's probability  - done")

## cellcycle clustering

pRb classes

In [ ]:
if config['isCellCycle']:
    c =  ['pRb-','pRb+']
    # pRb = Series([c[i] for i in np.random.randint(0,len(c),len(df))],index = df.index)#random classes for testing

    pRb = clustering.pickle_load(f"Labels_samp_arcsinh{config['j']}_CellCycle_Based_", config['dir_data'])
    pRb = Series([c[i] for i in pRb])
    del c
    print(pRb)


pRb vs clusters percentage and class vs pRb percentage (inverse)


In [ ]:

    
if config['isCellCycle']:

    cluster_samples_probability(pRb,labels,config,fname = 'pRb_clusters_probability',cluster_type_name='')
    print("pRb clusters's probability  - done")

    cluster_samples_probability(labels,pRb,config,fname = 'clusters_pRb_probability',cluster_type_name='cluster')
    print("clusters pRb's probability  - done")


pRb vs class percentage and class vs pRb percentage (inverse)


In [ ]:
if config['isCellCycle']:

    cluster_samples_probability(pRb,classes,config,fname = 'pRb_classes_probability',cluster_type_name='')
    print("pRb class's probability  - done")

    cluster_samples_probability(classes,pRb,config,fname = 'classes_pRb_probability',cluster_type_name='class')
    print("class pRb's probability  - done")

pRb in umap


In [ ]:
if config['isCellCycle']:

    plot.labeled_umap(umapData,pRb.copy(),colors = class_colors(),
                        title=config['title']+f'UMAP - by pRb',
                        figname='4_'+config['figname']+f'UMAP_by_pRb')


    

heatmap pRb

In [ ]:
if config['isCellCycle']:
    for features_group in heatmaps.features_groups:# CellCycle features are present only in the '.2' data other wise only epigenetic and cellcycle features
        heatmaps.plot(  df.copy(), pRb.copy(),
                        features = config['groups'][features_group].copy(),
                        ind = list(df.index[labels != -1]),#remove unclustered -1
                        title='HeatMap clusters -' + config['title'] + ': '+features_group,
                        figname='3_HeatMap_pRb_'+config['figname']+'using_'+features_group,)
        
    print('HeatMap by clusters - done')   


In [ ]:
# %run cellcycle.ipynb

## XGBClassifier fit, calculate metrics & shap

In [ ]:
from metrics_and_dataset_utils import *  
from xg_shap import Shap, fitXGBClassifier


by cluster - currently not very interesting

split into datasets - methods

In [ ]:
X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),samples.copy())
# X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes0.copy(),samples.copy(),train_samples = [  5,11,13,15,17,18,19,20],test_samples = [4,7,8,14] )
# X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),samples.copy(),samples.copy(),train_samples = [4.1],test_samples = [  4])

create new classes: 
1. classes - ['Unknown' 'Noise' 'Cycling' 'Luminal' 'Basal-like']
2. classes0 - ['Cycling' 'Luminal' 'Basal-like']
2. classes1 - ['Cycling' 'Lum&Basal']
3. classes2 - ['Luminal' 'Basal-like']

In [ ]:
# classes0 = classes[('Basal-like' in classes.)].copy()

In [ ]:

classes0 = classes[(classes=='Basal-like')|('Luminal' in classes)|(classes=='Cycling')].copy()
classes1 = classes[(classes=='Basal-like')|(classes=='Luminal')|(classes=='Cycling')].copy()
classes1[(classes=='Basal-like')|(classes=='Luminal')] = 'Lum&Basal'
classes2 = classes[(classes=='Basal-like')|(classes=='Luminal')].copy()
print(sorted(classes.unique()))
print(sorted(classes0.unique()))
print(sorted(classes1.unique()))
print(sorted(classes2.unique()))

original classes (all 3 & noise and unknown); XGBClassifier fit, calculate metrics & shap

In [ ]:
if len(classes0.unique())>1: #more than 1 class
    # drop df rows without label (drop points without label) and split randomly
    X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes0.copy(),LEN = 5000)
    # X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes0.copy(),samples.copy(),train_samples = [  5,11,13,15,17,18,19,20],test_samples = [4,7,8,14] )
    model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

    # predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
    accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                    .plot_all(config['dir_plots'] +'5_'+ config['figname']+'using_'+'all_classes_')
    print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")
    Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                            figname = '5_Shap_' + config['figname']+'using_'+'all_classes_')

Lum&Basal_vs_Cycling

In [ ]:
if len(classes1.unique())>1: #more than 1 class
    # drop df rows without label (drop points without label) and split randomly
    X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes1.copy(),LEN=5000)

    model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

    # predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
    accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                    .plot_all(config['dir_plots'] + '5_'+config['figname']+'using_'+'Lum&Basal_vs_Cycling_')

    Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                            figname = '5_Shap_' + config['figname']+'using_'+'Lum&Basal_vs_Cycling_')
    print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")


Lum_vs_Basal

In [ ]:
if len(classes2.unique())>1: #more than 1 class
    # drop df rows without label (drop points without label) and split randomly
    X_train, X_test, y_train, y_test = createDataset(df[config['features']].copy(),classes2.copy(),LEN=5000)

    model,label_encoder = fitXGBClassifier(X_train.copy(), y_train.copy())

    # predict y_test_pred using X_test and inverse transform it back to the original class labels - compare y_test_pred results to y_test
    accuracy = ClassificationMetrics(y_test,    Series(label_encoder.inverse_transform(model.predict(X_test))))\
                                    .plot_all(config['dir_plots'] + '5_'+config['figname']+'using_'+'Lum_vs_Basal_')

    Shap(**config).calculate(X_test,model,label_encoder,accuracy,colors = class_colors(),LEN = 1000,
                            figname = '5_Shap_' + config['figname']+'using_'+'Lum_vs_Basal_')
    print(f"classes accuracy {config['j']}_{config['feautures_ind']}: {accuracy}%")


## features analysis


plot features on umap 

In [ ]:
cols = config['groups']['NamesAll'].copy()
# add non-mutual features to specific samples
[(cols.append(feature) if (feature not in cols) and (feature in df.columns) else None)       for feature in ['CyclinB1', 'pRB','H3S28p','DNA2']]
        

# limits = np.hstack((np.floor([2*x for x in [umapData[:,0].min(),umapData[:,1].min()]])/2,\
#                     np.ceil([2*x for x in [umapData[:,0].max(),umapData[:,1].max()]])/2))
plot.umap_by_feature(umapData.copy(),df.copy(), cols, #df.copy().loc[ind], config,#limits = limits,
        title='UMAP ' + config["title"],
        figname='2_'+config["figname"]+'UMAP_')

plot.umap_by_feature(umapData.copy(),df.copy(), cols, ind = list(df.index[labels != -1]),#remove unclustered -1
                        #df.copy().loc[ind], config,#limits = limits,
        title='UMAP without -1' + config["title"],
        figname='2_'+config["figname"]+'UMAP_without_unclustered(-1)_')
del cols




scatter figures


In [ ]:
# if config['visualize']:
#     try:
#         scatter(k, 'MBD', 'DNA2', title, figname, settings)
#     except:
#         print('wanted features not in sample')
# # k.columns

## finalize

In [ ]:

# imList2pdf(dir_plots,j,['CellIden','EpiCols','CellIden+EpiCols'])
